In [1]:
import os
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import Xception
from sklearn.metrics import classification_report
from tensorflow.keras import Model

In [ ]:
train_data = pd.read_csv("/kaggle/input/140k-real-and-fake-faces/train.csv")
test_data  = pd.read_csv("/kaggle/input/140k-real-and-fake-faces/test.csv")
validation_data = pd.read_csv("/kaggle/input/140k-real-and-fake-faces/valid.csv")

In [ ]:
path="/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake"

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255.,preprocessing_function=preprocess_input)

train_data_generator = train_image_generator.flow_from_dataframe(
    dataframe=train_data,
    directory=path,
    x_col ='path',
    y_col ='label_str',
    color_mode="rgb",
    target_size=(256, 256),
    class_mode="binary",
    batch_size=32,
    shuffle = True
)


In [ ]:
validation_image_generator = ImageDataGenerator(rescale=1./255.,preprocessing_function=preprocess_input)
test_image_generator = ImageDataGenerator(rescale=1./255.,preprocessing_function=preprocess_input)

validation_data_generator = validation_image_generator.flow_from_dataframe(
    dataframe=validation_data,
    directory=path,
    x_col ='path',
    y_col ='label_str',
    color_mode="rgb",
    target_size=(256, 256),
    class_mode="binary",
    batch_size=32,
    shuffle = True
)


test_data_generator = test_image_generator.flow_from_dataframe(
    dataframe=test_data,
    directory=path,
    x_col ='path',
    y_col ='label_str',
    color_mode="rgb",
    target_size=(256, 256),
    class_mode="binary",
    batch_size=32,
    shuffle = False
)


In [ ]:
labels = train_data_generator.class_indices
class_names = list(labels.keys())
print('class names:',class_names)

In [ ]:
import matplotlib.pyplot as plt

labels = ["Train", "Validation", "Test"]
counts = [len(train_data), len(validation_data), len(test_data)]

fig, ax = plt.subplots()
ax.pie(counts, labels=labels, autopct="%1.1f%%")
ax.set_title("Distribution of Images")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

train_size = len(train_data)
validation_size = len(validation_data)
test_size = len(test_data)

fig, ax = plt.subplots()
x = ['Train', 'Validation', 'Test']
y = [train_size, validation_size, test_size]
ax.bar(x, y)
ax.set_xlabel('Dataset')
ax.set_ylabel('Number of Images')
ax.set_title('Distribution of Images in Each Dataset')

plt.show()

In [ ]:
base_model = Xception(weights='imagenet',include_top=False,input_shape=(256, 256, 3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

Xception_model = Sequential()
Xception_model.add(base_model)
Xception_model.add(Flatten())
Xception_model.add(BatchNormalization())
Xception_model.add(Dense(256, activation='relu'))
Xception_model.add(Dropout(0.5))
Xception_model.add(Dense(128, activation='relu'))
Xception_model.add(Dropout(0.5))
Xception_model.add(BatchNormalization())
Xception_model.add(Dense(1, activation='sigmoid'))

Xception_model.summary()

In [ ]:
Xception_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = Xception_model.fit(train_data_generator ,epochs = 10,validation_data= (validation_data_generator))
print(history.history)

In [ ]:
test_loss, test_accuracy = Xception_model.evaluate(test_data_generator)
print(f'Loss: {test_loss}, Accuracy: {test_accuracy}')

In [ ]:
# Save your model
Xception_model.save('/Deep Img/model_xxeption.keras')

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

predictions = Xception_model.predict(test_data_generator)
predicted_labels = np.where(predictions > 0.5, 1, 0)
true_labels = test_data_generator.classes
conf_matrix = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_data_generator.class_indices,
            yticklabels=test_data_generator.class_indices)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(true_labels, predicted_labels, target_names=class_names)

print("Classification Report:")
print(report)
